### Requirements/Libraries


In [1]:
import os
import pandas as pd
from tqdm import tqdm
from collections import Counter
from SPARQLWrapper import SPARQLWrapper, JSON


Read CSV files (tables)

In [2]:
dir = os.path.join('.', 'WikidataTables2024R1', 'DataSets', 'Valid', 'tables')
table_paths = os.listdir(dir)
table_paths = [os.path.abspath(os.path.join(dir, p)) for p in table_paths]
tables = {}
for table_path in table_paths:
    table_name, _ = os.path.splitext(os.path.basename(table_path))
    df = pd.read_csv(table_path, sep=',', quotechar='"',escapechar="\\")
    tables[table_name] = df

Querying functions for wikidata

In [7]:
def query_wikidata(query):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results


def get_wikidata_entity(label):
    query = f"""
    SELECT ?entity ?entityLabel WHERE {{
      ?entity ?label "{label}"@en.
      FILTER STRSTARTS(STR(?entity), "http://www.wikidata.org/entity/")
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }} LIMIT 10
    """
    results = query_wikidata(query)
    return results['results']['bindings']


def get_column_type(column_values):
    # Convert column values to strings and ensure proper escaping of quotes
    column_values = [str(value).replace('"', '\\"') for value in column_values]
    values_str = ' '.join([f'"{value}"@en' for value in column_values])
    
    # Construct the SPARQL query
    query = f"""
    SELECT ?type ?typeLabel WHERE {{
      VALUES ?value {{ {values_str} }}
      ?entity rdfs:label ?value.
      ?entity wdt:P31 ?type.
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }} LIMIT 100
    """
    
    results = query_wikidata(query)
    return results['results']['bindings']

Simple disambiguation techniques

In [8]:
def disambiguate_entities(entities):
    entity_labels = [entity['entity']['value'] for entity in entities]
    most_common_entity = Counter(entity_labels).most_common(1)[0][0]
    return most_common_entity

def disambiguate_column_type(types):
    type_labels = [t['type']['value'] for t in types]
    most_common_type = Counter(type_labels).most_common(1)[0][0]
    return most_common_type

Matching functions

In [9]:
# Match cells to entities
def match_cells_to_entities(df):
    matched_entities = []
    for row_id, row in df.iterrows():
        for column_id, cell in enumerate(row):
            if pd.notna(cell):
                entities = get_wikidata_entity(cell)
                if entities:
                    matched_entity = disambiguate_entities(entities)
                    matched_entities.append((row_id, column_id, matched_entity))
    return matched_entities


# Match columns to types
def match_columns_to_types(df):
    matched_types = {}
    for column_id, column in enumerate(df.columns):
        column_values = df[column].dropna().unique()
        types = get_column_type(column_values)
        if types:
            matched_type = disambiguate_column_type(types)
            matched_types[column_id] = matched_type
    return matched_types

Process multiple CSV files and save results

In [11]:
results_cea, results_cta = [], []

for table_name, df in tqdm(tables.items()):
    try:
        matched_entities = match_cells_to_entities(df)
        matched_types = match_columns_to_types(df)
    except:
        # print(f"Error in table {table_name}")
        continue
    for row_id, column_id, entity in matched_entities:
        results_cea.append([table_name, row_id, column_id, entity])
    for column_id, entity in matched_types.items():
        results_cta.append([table_name, column_id, entity])

# transform results to a dataframe
cea_df = pd.DataFrame(results_cea, columns=['Column1', 'Column2', 'Column3', 'Column4'])
cta_df = pd.DataFrame(results_cta, columns=['Column1', 'Column2', 'Column3'])

# save the results to the targets directory
dir = os.path.join('.', 'WikidataTables2024R1', 'DataSets', 'Valid')
cea_df.to_csv(os.path.join(dir, 'targets', 'cea_targets.csv'), index=False)
cta_df.to_csv(os.path.join(dir, 'targets', 'cta_targets.csv'), index=False)

100%|██████████| 500/500 [22:43<00:00,  2.73s/it]


Evaluation

In [12]:
from CEA_Evaluator import CEA_Evaluator
from CTA_Evaluator import CTA_Evaluator


gt_file = os.path.join(dir, 'gt', 'cea_gt.csv')
system_file = os.path.join(dir, 'targets', 'cea_targets.csv')

# Instantiate an evaluator
evaluator = CEA_Evaluator()
# Evaluate
result = evaluator._evaluate(system_file, gt_file)
print("CEA results: ")
print(result)

gt_file = os.path.join(dir, 'gt', 'cta_gt.csv')
system_file = os.path.join(dir, 'targets', 'cta_targets.csv')

# Instantiate an evaluator
evaluator = CTA_Evaluator()
# Evaluate
result = evaluator._evaluate(system_file, gt_file)
print("CTA results: ")
print(result)

0.047 0.060 0.038
CEA results: 
{'score': 0.04690647482014388, 'score_secondary': 0.060303366629670736}
0.607 0.671 0.554
CTA results: 
{'score': 0.6068601583113457, 'score_secondary': 0.6712062256809338}


In [ ]:
# def print_matched_results(matched_entities, matched_types):
#     print("Matched Entities:")
#     for cell, entity in matched_entities.items():
#         print(f"{cell} -> {entity}")
    
#     print("\nMatched Column Types:")
#     for column, column_type in matched_types.items():
#         print(f"{column} -> {column_type}")

# print_matched_results(matched_entities, matched_types)